# Big Data Analytics Project 

In [5]:
import pandas as pd

# Load the JSON file into a Pandas DataFrame
salesdf = pd.read_json('sales.json', lines=True)


In [6]:
!pip install pandas

In [7]:
num_rows, num_columns = salesdf.shape

print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 5000
Number of Columns: 7


In [14]:
from IPython.display import display, HTML

# Assuming 'salesdf' is your DataFrame
html_table = salesdf.head(3).to_html(classes='customtable', index=False)

# Define custom CSS styles to color the header
styles = """
    <style>
        .customtable {
            border-collapse: collapse;
            width: 100%;
        }
        .customtable th {
            background-color: #4e20ce; /* background color for header */
            color: white; /* White text color for header */
            border: 1px solid black;
            padding: 8px;
            text-align: left;
        }
        .customtable td {
            border: 1px solid black;
            padding: 8px;
            text-align: left;
        }
    </style>
"""

# Display the HTML with custom styles
display(HTML(styles + html_table))


_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},"[{'name': 'printer paper', 'tags': ['office', 'stationary'], 'price': {'$numberDecimal': '40.01'}, 'quantity': 2}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '35.29'}, 'quantity': 2}, {'name': 'pens', 'tags': ['writing', 'office', 'school', 'stationary'], 'price': {'$numberDecimal': '56.12'}, 'quantity': 5}, {'name': 'backpack', 'tags': ['school', 'travel', 'kids'], 'price': {'$numberDecimal': '77.71'}, 'quantity': 2}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '18.47'}, 'quantity': 2}, {'name': 'envelopes', 'tags': ['stationary', 'office', 'general'], 'price': {'$numberDecimal': '19.95'}, 'quantity': 8}, {'name': 'envelopes', 'tags': ['stationary', 'office', 'general'], 'price': {'$numberDecimal': '8.08'}, 'quantity': 3}, {'name': 'binder', 'tags': ['school', 'general', 'organization'], 'price': {'$numberDecimal': '14.16'}, 'quantity': 3}]",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@witwuta.sv', 'satisfaction': 4}",True,Online
{'$oid': '5bd761dcae323e45a93ccfe9'},{'$date': '2015-08-25T10:01:02.918Z'},"[{'name': 'envelopes', 'tags': ['stationary', 'office', 'general'], 'price': {'$numberDecimal': '8.05'}, 'quantity': 10}, {'name': 'binder', 'tags': ['school', 'general', 'organization'], 'price': {'$numberDecimal': '28.31'}, 'quantity': 9}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '20.95'}, 'quantity': 3}, {'name': 'laptop', 'tags': ['electronics', 'school', 'office'], 'price': {'$numberDecimal': '866.5'}, 'quantity': 4}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '33.09'}, 'quantity': 4}, {'name': 'printer paper', 'tags': ['office', 'stationary'], 'price': {'$numberDecimal': '37.55'}, 'quantity': 1}, {'name': 'backpack', 'tags': ['school', 'travel', 'kids'], 'price': {'$numberDecimal': '83.28'}, 'quantity': 2}, {'name': 'pens', 'tags': ['writing', 'office', 'school', 'stationary'], 'price': {'$numberDecimal': '42.9'}, 'quantity': 4}, {'name': 'envelopes', 'tags': ['stationary', 'office', 'general'], 'price': {'$numberDecimal': '16.68'}, 'quantity': 2}]",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@hem.uy', 'satisfaction': 5}",False,Phone
{'$oid': '5bd761dcae323e45a93ccfea'},{'$date': '2017-06-22T09:54:14.185Z'},"[{'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '30.08'}, 'quantity': 1}, {'name': 'binder', 'tags': ['school', 'general', 'organization'], 'price': {'$numberDecimal': '24.68'}, 'quantity': 7}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '32.1'}, 'quantity': 4}, {'name': 'pens', 'tags': ['writing', 'office', 'school', 'stationary'], 'price': {'$numberDecimal': '31.41'}, 'quantity': 3}, {'name': 'printer paper', 'tags': ['office', 'stationary'], 'price': {'$numberDecimal': '43.56'}, 'quantity': 3}, {'name': 'pens', 'tags': ['writing', 'office', 'school', 'stationary'], 'price': {'$numberDecimal': '68.95'}, 'quantity': 4}, {'name': 'notepad', 'tags': ['office', 'writing', 'school'], 'price': {'$numberDecimal': '8.57'}, 'quantity': 4}, {'name': 'backpack', 'tags': ['school', 'travel', 'kids'], 'price': {'$numberDecimal': '106.6'}, 'quantity': 2}, {'name': 'laptop', 'tags': ['electronics', 'school', 'office'], 'price': {'$numberDecimal': '596.33'}, 'quantity': 4}]",Denver,"{'gender': 'M', 'age': 51, 'email': 'worbiduh@vowbu.cg', 'satisfaction': 5}",False,In store


In [15]:

# Extract relevant information from the 'items' column
items_df = pd.json_normalize(salesdf['items'].explode()).reset_index()

In [19]:

# Assuming 'items_df' is another DataFrame
head_rows = items_df.head(3)
tail_rows = items_df.tail(3)

# Add a row with "..." to indicate skipped rows
ellipsis_row = pd.DataFrame(["..."] * len(head_rows.columns)).T
ellipsis_row.columns = head_rows.columns
table_with_ellipsis = pd.concat([head_rows, ellipsis_row, tail_rows])

# Convert to HTML
html_table = table_with_ellipsis.head(5).to_html(classes='customtable', index=False)

# Display the HTML with custom styles for items_df
display(HTML(styles + html_table))

index,name,tags,quantity,price.$numberDecimal
0,printer paper,"[office, stationary]",2,40.01
1,notepad,"[office, writing, school]",2,35.29
2,pens,"[writing, office, school, stationary]",5,56.12
...,...,...,...,...
27435,binder,"[school, general, organization]",3,29.09


In [21]:
# Assuming 'items_df' is your DataFrame
items_df['sales'] = items_df['quantity'] * items_df['price.$numberDecimal'].astype(float)

# Group by product name and calculate total sales
Top_10_Product_Sales = items_df[['name','sales']].sort_values(by='sales', ascending=False).head(10)

# Add a new column 'rank' with numbers (1, 2, 3, ...)
Top_10_Product_Sales.insert(0, 'Top10', range(1, len(Top_10_Product_Sales) + 1))

# Convert to HTML
html_table = Top_10_Product_Sales.head(10).to_html(classes='customtable', index=False)

# Display the HTML with custom styles for Top_10_Product_Sales
display(HTML(styles + html_table))

Top10,name,sales
1,laptop,7966.55
2,laptop,7933.50
3,laptop,7912.50
4,laptop,7888.10
5,laptop,7883.50
6,laptop,7878.80
7,laptop,7877.55
8,laptop,7863.15
9,laptop,7852.45
10,laptop,7844.90


# 2.Show Top 3 Products (name) Sales by  Store(location)

In [22]:
salesbystore = salesdf[['storeLocation','purchaseMethod','saleDate']].reset_index()

In [23]:
merged_df = pd.merge(items_df,salesbystore, left_on='index', right_on='index')
html_table = merged_df.head().to_html(classes='customtable', index=False)
display(HTML(styles + html_table))


index,name,tags,quantity,price.$numberDecimal,sales,storeLocation,purchaseMethod,saleDate
0,printer paper,"[office, stationary]",2,40.01,80.02,Denver,Online,{'$date': '2015-03-23T21:06:49.506Z'}
1,notepad,"[office, writing, school]",2,35.29,70.58,Seattle,Phone,{'$date': '2015-08-25T10:01:02.918Z'}
2,pens,"[writing, office, school, stationary]",5,56.12,280.60,Denver,In store,{'$date': '2017-06-22T09:54:14.185Z'}
3,backpack,"[school, travel, kids]",2,77.71,155.42,Seattle,In store,{'$date': '2015-02-23T09:53:59.343Z'}
4,notepad,"[office, writing, school]",2,18.47,36.94,London,In store,{'$date': '2017-12-03T18:39:48.253Z'}


In [24]:
# Assuming 'merged_df' is your DataFrame
top_3_productsales_by_store = merged_df.groupby(['storeLocation', 'name'])['sales'].sum().reset_index()
top_3_productsales_by_store = top_3_productsales_by_store.sort_values(by='sales', ascending=False)

# Add a new column 'rank' with numbers (1, 2, 3, ...)
top_3_productsales_by_store.insert(0, 'Top3', range(1, len(top_3_productsales_by_store) + 1))

# Convert to HTML
html_table = top_3_productsales_by_store.head(3).to_html(classes='customtable', index=False)

# Display the HTML with custom styles for top_3_productsales_by_store
display(HTML(styles + html_table))

Top3,storeLocation,name,sales
1,Denver,laptop,409752.19
2,Seattle,laptop,237711.23
3,London,laptop,201480.89


# 3.Show rankings of each store (location)

In [25]:
# Group by store location and calculate total sales
store_rankings = merged_df.groupby('storeLocation')['sales'].sum().reset_index()
# Convert to HTML
html_table = store_rankings.head(3).to_html(classes='customtable', index=False)

# Display the HTML with custom styles for top_3_productsales_by_store
display(HTML(styles + html_table))

storeLocation,sales
Austin,237868.30
Denver,583256.15
London,291162.89


In [26]:
# Sort the values by total sales in descending order
store_rankings = store_rankings.sort_values(by='sales', ascending=False)

# Add a rank column based on total sales
store_rankings['rank'] = range(1, len(store_rankings) + 1)

# Convert to HTML
html_table = store_rankings.head().to_html(classes='customtable', index=False)

# Display the HTML with custom styles for top_3_productsales_by_store
display(HTML(styles + html_table))

storeLocation,sales,rank
Denver,583256.15,1
Seattle,370600.53,2
London,291162.89,3
Austin,237868.30,4
New York,233406.01,5


# 4.Show purchased method by gender table 

In [27]:
print(merged_df['purchaseMethod'].unique())

['Online' 'Phone' 'In store']


In [34]:

# Assuming customer information is in a column named 'customer'
customer_df = salesdf['customer'].apply(pd.Series)
# Merge the customer information with the items information
merged_customer_df = pd.merge(merged_df, customer_df, left_on='index', right_index=True)
# Convert to HTML
html_table = merged_customer_df.head().to_html(classes='customtable', index=False)

# Display the HTML with custom styles for top_3_productsales_by_store
display(HTML(styles + html_table))

index,name,tags,quantity,price.$numberDecimal,sales,storeLocation,purchaseMethod,saleDate,gender,age,email,satisfaction
0,printer paper,"[office, stationary]",2,40.01,80.02,Denver,Online,{'$date': '2015-03-23T21:06:49.506Z'},M,42,cauho@witwuta.sv,4
1,notepad,"[office, writing, school]",2,35.29,70.58,Seattle,Phone,{'$date': '2015-08-25T10:01:02.918Z'},M,50,keecade@hem.uy,5
2,pens,"[writing, office, school, stationary]",5,56.12,280.60,Denver,In store,{'$date': '2017-06-22T09:54:14.185Z'},M,51,worbiduh@vowbu.cg,5
3,backpack,"[school, travel, kids]",2,77.71,155.42,Seattle,In store,{'$date': '2015-02-23T09:53:59.343Z'},F,45,vatires@ta.pe,3
4,notepad,"[office, writing, school]",2,18.47,36.94,London,In store,{'$date': '2017-12-03T18:39:48.253Z'},M,40,dotzu@ib.sh,4


In [39]:
# Assuming 'purchaseMethod' is a column in your dataframe
purchase_by_gender = pd.crosstab(merged_customer_df['gender'], merged_customer_df['purchaseMethod'])

# Display the table
print(purchase_by_gender)


purchaseMethod  In store  Online  Phone
gender                                 
F                   1430     813    284
M                   1389     772    312


In [55]:

# Assuming 'saleDate' is the column containing date information
merged_customer_df['saleDate'] = merged_customer_df['saleDate'].apply(lambda x: x['$date'] if isinstance(x, dict) else x)

# Convert 'saleDate' to datetime format
merged_customer_df['saleDate'] = pd.to_datetime(merged_customer_df['saleDate'])

# Extracting year and month from 'saleDate'
merged_customer_df['year'] = merged_customer_df['saleDate'].dt.year
merged_customer_df['month'] = merged_customer_df['saleDate'].dt.month

# Drop rows with non-finite (NA or inf) values in 'month'
merged_customer_df = merged_customer_df.dropna(subset=['month'])

# Convert 'month' and 'year' to integer type
merged_customer_df['month'] = merged_customer_df['month'].astype(int)
merged_customer_df['year'] = merged_customer_df['year'].astype(int)

# Assuming 'sales' is the column representing sales amount
# If it's not present, you may need to check how the sales data is stored in your DataFrame
monthly_total_sales = merged_customer_df.groupby(['year', 'month']).agg({'sales': 'sum'}).reset_index()

# Convert to HTML
html_table = monthly_total_sales.to_html(classes='customtable', index=False)

# Display the HTML with custom styles for top_3_productsales_by_store
display(HTML(styles + html_table))



year,month,sales
2013,1,22516.78
2013,2,29306.27
2013,3,19943.84
2013,4,38496.15
2013,5,27018.06
2013,6,33785.74
2013,7,25126.67
2013,8,20167.53
2013,9,27709.83
2013,10,29965.56
